Import requests library to query the yahoo finance website and get the ticket symbol.
Import fuzzywuzzy to get a partial match against yahoo's website.
Import pandas_datareader to get the close date for each ticker symbol that we found.

In [1]:
import requests as rq
import pandas as pd
import fuzzywuzzy as fw
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas_datareader as pdr

/usr/local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Read in the excel file from the Information is Beautiful website

In [2]:
data_breach = pd.read_excel('Information is Beautiful- Data Breaches (public).xlsx')

Drop the first row which has the descriptions

In [3]:
data_breach = data_breach.drop(data_breach.index[[0]])

Bring in only the rows that I need from the spreadsheet

In [4]:
data_breach = data_breach[['Entity','YEAR','records lost','ORGANISATION','METHOD OF LEAK','DATA SENSITIVITY']]

Drop any rows that have blank values

In [5]:
data_breach = data_breach.dropna(how='any') 

Below we'll replace the year values from 1-14 to the actual year. 14 is going to be 2017 because that is the most recent and not 2018 (which is the future)

In [6]:
data_breach['YEAR'] = data_breach['YEAR'].replace([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],[2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2017])
#replace year values

Below is the code needed to search our excel document with the yahoo API. Entities will take place of the squiggly brackets.
The 'if statement' will see if the partial ratio is greater than 80 (means the spreadsheet Entity matches at least 80 percent of what's in yahoo finance) and if the exchange is NYSE or NASDAQ. If it is, then it will return the ticket symbol.

In [7]:
def get_symbol(entity):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(entity)

    result = rq.get(url).json()

    for x in result['ResultSet']['Result']:
        if fuzz.partial_ratio(entity, x['name']) >= 80:
            if x['exchDisp'] == 'NYSE' or x['exchDisp'] == 'NASDAQ':
                return x['symbol'] 

Make the Entity column as a list

In [8]:
entities = data_breach['Entity'].tolist() #do if then statement to say if match > 90 then take name

Create an empty list which will be used to get the close dates

In [9]:
tickerList = []

Iterate through the get_symbol function for every entity in our spreadsheet. After you get the symbol then append it to the empty tickerList

In [10]:
for y in entities:
    symbol = get_symbol(y)
    tickerList.append(symbol)  

Not all symbols could be found through the Yahoo API. Error message said it couldn't read the URL, so I added a try-catch where if it errors out, then I'll print that the ticker wasn't found. 
Also I created an empty data frame with 3 columns and the for loop will iterate through and put all the data into the empty data frame, stockData.

In [11]:
stockData = pd.DataFrame(columns = ['Symbol','Date', 'Close'])
for x in tickerList:
    if x is not None:
        try:
            result = pdr.get_data_yahoo(x)
            result = result.reset_index()  
            result['Symbol'] = x
            result = result[['Symbol', 'Date', 'Close']]
            stockData = stockData.append(result)
        except:
            print('ticker not found')
    else:
        pass

ticker not found
ticker not found


Insert the ticker column into the end of the  original data_breach dataframe

In [12]:
data_breach.insert(loc = 6,column = 'ticker', value = tickerList)

Do a right join to merge the data breaches dataframe with the stockData dataframe. Match based on ticker column from data breach with Symbol from stockData

In [13]:
updated_data = pd.merge(data_breach, stockData, left_on='ticker',right_on='Symbol', how='right')

Export the dataframe as a csv to upload to Tableau

In [14]:
updated_data.to_csv('Lab_7_Cleaned_Data2.csv',index=False)